# 数据归档

## 配置信息

In [138]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import os
pymysql.install_as_MySQLdb()

# 修改当前主目录
os.chdir(r'C:/Users/yves1/Desktop/')

# SKU配置信息
sku_data = pd.read_json(r'D:\GitHub\Jupyter\DataSet\sku_data.json')

# 数据库配置信息
db_engine = str(r'mysql+mysqldb://%s:' + '%s' + '@%s/%s') % (sku_data.loc['USER', 'Data'], sku_data.loc['PWD', 'Data'], sku_data.loc['IP', 'Data'], sku_data.loc['DB', 'Data'])

# 表名信息
shark_gray_business_us = 'tbl_shark_gray_business_us'
shark_gray_ad_us = 'tbl_shark_gray_ad_us'

## 业务报表

In [140]:
def parse_business_report(date, table_name):
    business_report = pd.read_csv(next(value for value in os.listdir() if value.startswith('BusinessReport')))
    business_report.drop(['Title', 'Session Percentage', 'Page Views Percentage'], axis=1, inplace=True)
    business_report.rename({'(Child) ASIN': 'SKU'}, axis=1, inplace=True)
    business_report['SKU'] = business_report['SKU'].apply(lambda x: sku_data.loc[x.strip(), 'Data'])
    business_report.insert(loc=0, column='Date', value=pd.to_datetime(date, format='%Y/%m/%d'))
    engine = create_engine(db_engine)
    try:
        business_report.to_sql(table_name, engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    finally:
        engine.dispose()

## 广告报表

In [228]:
def parse_sponsored_report(date, table_name):
    sponsored_report = pd.read_excel(next(value for value in os.listdir() if value.startswith('Sponsored Products')))
    sponsored_report = sponsored_report[sponsored_report['Date']==date]
    sponsored_report.drop(['Currency', 'Ad Group Name', 'Total Return on Advertising Spend (RoAS)', '7 Day Advertised SKU Units (#)', '7 Day Other SKU Units (#)', '7 Day Advertised SKU Sales ', '7 Day Other SKU Sales '], axis=1, inplace=True)
    sponsored_report.rename({'Click-Thru Rate (CTR)': 'CTR', 'Cost Per Click (CPC)': 'CPC', '7 Day Total Sales ': 'Total Sales', 'Total Advertising Cost of Sales (ACoS) ': 'ACoS', '7 Day Total Orders (#)': 'Total Orders', '7 Day Total Units (#)': 'Total Units', '7 Day Conversion Rate': 'CR'}, axis=1, inplace=True)
    sponsored_report['ACoS'].fillna(value=0, inplace=True)
    sponsored_report[['CTR', 'CPC', 'ACoS', 'CR']] = sponsored_report[['CTR', 'CPC', 'ACoS', 'CR']].apply(lambda x: round(x, 4))
    sponsored_report.sort_values(by=['Portfolio name', 'Campaign Name'], inplace=True)
    engine = create_engine(db_engine)
    try:
        sponsored_report.to_sql(table_name, engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    finally:
        engine.dispose()

In [236]:
# 解析业务报表数据
# parse_business_report('2020/12/27', shark_gray_business_us)

# 解析赞助广告数据
# parse_sponsored_report('2020/12/29', shark_gray_ad_us)

# 数据统计

# 数据可视化